In [1]:
from tensorflow.keras import layers, models, losses, Model
import numpy as np
import tensorflow as tf

2025-01-26 17:49:38.888549: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-26 17:49:38.888870: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-26 17:49:38.890562: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-26 17:49:38.895481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737901178.903886 1235627 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737901178.90

In [56]:
import shutil
import os
# Paths
#test_augmentation
original_dir = "/home/mahdi/logo_detection_model/autoencoder/detected_logos/"
train_dir = "/home/mahdi/logo_detection_model/autoencoder/train"
val_dir = "/home/mahdi/logo_detection_model/autoencoder/validation"

# Split data into train and validation sets
for class_name in os.listdir(original_dir):
    class_path = os.path.join(original_dir, class_name)
    train_class_path = os.path.join(train_dir, class_name)
    val_class_path = os.path.join(val_dir, class_name)
    
    os.makedirs(train_class_path, exist_ok=True)
    os.makedirs(val_class_path, exist_ok=True)

    images = os.listdir(class_path)
    split_idx = int(len(images) * 0.8)  # 80% for training, 20% for validation

    for img_name in images[:split_idx]:
        shutil.copy(os.path.join(class_path, img_name), train_class_path)
    for img_name in images[split_idx:]:
        shutil.copy(os.path.join(class_path, img_name), val_class_path)

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/home/mahdi/logo_detection_model/autoencoder/train"
val_dir = "/home/mahdi/logo_detection_model/autoencoder/validation"
dir = "/home/mahdi/logo_detection_model/autoencoder/detected_logos/"


def safe_preprocess(img):
    try:
        return img  # Pass through valid images
    except Exception as e:
        print("Skipping corrupted image.")
        return np.zeros((224, 224, 3))  # Placeholder for invalid images


# Training Data Generator
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  
    
    preprocessing_function=safe_preprocess
)

# Validation Data Generator
val_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  
    
    preprocessing_function=safe_preprocess
)

# Create Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Path to training data
    target_size=(224, 224),  # Resize images
    batch_size=16,  # Batch size
    class_mode='input'  # <-- This is important for autoencoders
)

val_generator = val_datagen.flow_from_directory(
     val_dir,  # Path to validation data
    target_size=(224, 224),  # Resize images
    batch_size=16,  # Batch size
    class_mode='input'  # <-- This is important for autoencoders
)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")

Found 206 images belonging to 1 classes.
Found 52 images belonging to 1 classes.
Training samples: 206
Validation samples: 52


#### Using class_mode = 'input' for autoencoder

In [3]:
print(f"Train generator samples: {train_generator.samples}")
print(f"Validation generator samples: {val_generator.samples}")


Train generator samples: 206
Validation generator samples: 52


In [4]:
# Try fetching a batch of data
batch = next(iter(train_generator))
print("Train batch shapes:", [x.shape for x in batch])


Train batch shapes: [(16, 224, 224, 3), (16, 224, 224, 3)]


/home/mahdi/logo_detection_model/.venv/lib/python3.12/site-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [5]:
# Try fetching a batch of data
batch = next(iter(val_generator))
print("val batch shapes:", [x.shape for x in batch])

val batch shapes: [(16, 224, 224, 3), (16, 224, 224, 3)]


In [61]:
# batch = next(iter(val_generator))
# print(f"Input batch shape: {batch.shape}")  # Should match (batch_size, 224, 224, 3)


In [62]:
len(train_generator)

13

In [63]:
len(val_generator)

4

In [83]:

# # Autoencoder architecture
# def build_autoencoder(input_shape=(224, 224, 3), latent_dim=128):
#     # Encoder
#     input_img = layers.Input(shape=input_shape)
#     x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
#     x = layers.MaxPooling2D((2, 2), padding="same")(x)
#     x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
#     x = layers.MaxPooling2D((2, 2), padding="same")(x)
#     x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
#     x = layers.Flatten()(x)
#     latent = layers.Dense(latent_dim, activation="relu")(x)  # Latent space

#     # Decoder
#     x = layers.Dense(28 * 28 * 128, activation="relu")(latent)
#     x = layers.Reshape((28, 28, 128))(x)
#     x = layers.Conv2DTranspose(64, (3, 3), activation="relu", padding="same")(x)
#     x = layers.UpSampling2D((2, 2))(x)
#     x = layers.Conv2DTranspose(32, (3, 3), activation="relu", padding="same")(x)
#     x = layers.UpSampling2D((2, 2))(x)
#     output_img = layers.Conv2DTranspose(3, (3, 3), activation="sigmoid", padding="same")(x)

#     # Autoencoder Model
#     autoencoder = models.Model(input_img, output_img)
#     encoder = models.Model(input_img, latent)  # Separate encoder model for feature extraction

#     return autoencoder, encoder



In [6]:
from tensorflow.keras import layers, models

# Encoder
encoder_input = layers.Input(shape=(224, 224, 3))
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(encoder_input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)  # Down to (112, 112, 64)
x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)  # Down to (56, 56, 128)
encoder_output = x

# Decoder
x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(encoder_output)
x = layers.UpSampling2D((2, 2))(x)  # Up to (112, 112, 128)
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = layers.UpSampling2D((2, 2))(x)  # Up to (224, 224, 64)
decoder_output = layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)  # Match input shape

# Autoencoder Model
# decoder_output = layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(decoder_output)
# autoencoder = Model(encoder_input, decoder_output)
autoencoder = models.Model(encoder_input, decoder_output)
autoencoder.compile(optimizer="adam", loss="mse")

autoencoder.summary()


W0000 00:00:1737901207.969922 1235627 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 112, 112, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 224, 224, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 224, 224, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 298,755 (1.14 MB)

 Trainable params: 298,755 (1.14 MB)

 Non-trainable params: 0 (0.00 B)

In [114]:
# # Build and compile the autoencoder
# autoencoder, encoder = build_autoencoder()
# autoencoder.compile(optimizer="adam", loss=losses.MeanSquaredError())

In [65]:
print("Autoencoder input shape:", autoencoder.input_shape)
print("Autoencoder output shape:", autoencoder.output_shape)


Autoencoder input shape: (None, 224, 224, 3)
Autoencoder output shape: (None, 224, 224, 3)


In [66]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = val_generator.samples // val_generator.batch_size
print(f"Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}")


Steps per epoch: 12, Validation steps: 3


In [10]:
# Check the generators
assert train_generator.samples > 0, "Training directory is empty or misconfigured!"
assert val_generator.samples > 0, "Validation directory is empty or misconfigured!"

steps_per_epoch = max(1, train_generator.samples // train_generator.batch_size)
validation_steps = max(1, val_generator.samples // val_generator.batch_size)


## Test to debug Error: None values not supported.

In [36]:
# Debug the data generators
batch = next(iter(train_generator))
print(f"Batch shape: {batch.shape if batch is not None else 'None'}")


Batch shape: (13, 224, 224, 3)


In [37]:
steps_per_epoch = max(1, train_generator.samples // train_generator.batch_size)
validation_steps = max(1, val_generator.samples // val_generator.batch_size)

print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")


Steps per epoch: 1
Validation steps: 1


In [39]:
batch = next(iter(train_generator))
print(f"Train batch shape: {batch.shape}")


Train batch shape: (13, 224, 224, 3)


In [40]:
batch = next(iter(val_generator))
print(f"Validation batch shape: {batch.shape}")


Validation batch shape: (7, 224, 224, 3)


/home/mahdi/logo_detection_model/.venv/lib/python3.12/site-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [68]:
# predictions = autoencoder.predict(batch)
# print(f"Prediction shape: {predictions.shape}")


In [43]:
batch = next(iter(train_generator))
predictions = autoencoder(batch)
print(f"Prediction shape: {predictions.shape}")


Prediction shape: (13, 224, 224, 3)


## Run the Model

In [7]:
history = autoencoder.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    steps_per_epoch=max(1, train_generator.samples // train_generator.batch_size),
    validation_steps=max(1, val_generator.samples // val_generator.batch_size),
    verbose=1
)


Epoch 1/50


/home/mahdi/logo_detection_model/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2025-01-26 17:50:22.444014: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT32 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 354ms/step - loss: 0.1335 - val_loss: 0.0531
Epoch 2/50
 1/12 ━━━━━━━━━━━━━━━━━━━━ 3s 298ms/step - loss: 0.0567

/home/mahdi/logo_detection_model/.venv/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0567 - val_loss: 0.0452
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 342ms/step - loss: 0.0434 - val_loss: 0.0322
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0314 - val_loss: 0.0325
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 345ms/step - loss: 0.0274 - val_loss: 0.0283
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0296 - val_loss: 0.0194
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 342ms/step - loss: 0.0191 - val_loss: 0.0157
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0154 - val_loss: 0.0179
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 349ms/step - loss: 0.0144 - val_loss: 0.0136
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0116 - val_loss: 0.0123
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 343ms/step - loss: 0.0117 - val_loss: 0.0122
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0109 - val_loss: 0.0120
Epoch 13/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 343ms/step - loss: 0.0105 

In [9]:
# Save the encoder for feature extraction
autoencoder.save("logo_autoencoder.h5")


## similarity part

### choosing Threshold

In [14]:
import numpy as np

valid_img = ['BM_LOGO-00.png' , 'BM_LOGO-01.png' ,  'BM_LOGO-02.png' , 'BM_LOGO-03.png' , 'BM_LOGO-04.png','BM_LOGO-05.png']
valid_img_path = '/home/mahdi/logo_detection_model/autoencoder/Valid_images'
# Prepare test generator
test_datagen = ImageDataGenerator(rescale=1.0 / 255)  
test_generator = test_datagen.flow_from_directory(
    directory=valid_img_path,  
    target_size=(224, 224),        
    batch_size=2,                 
    class_mode="input",            
    shuffle=False                   
)


Found 6 images belonging to 1 classes.


In [12]:
import numpy as np

test_generator.reset()
all_errors = []

# Go through each batch
for i in range(len(test_generator)):
    x_batch, _ = test_generator[i]  # shape (batch_size, 224, 224, 3)
    reconstructed_batch = autoencoder.predict(x_batch)
    
    # Compute errors for this batch
    batch_errors = np.mean(np.square(x_batch - reconstructed_batch), axis=(1,2,3))
    all_errors.append(batch_errors)

# Concatenate all batch errors
errors_val = np.concatenate(all_errors, axis=0)
print("Reconstruction errors shape:", errors_val.shape)
print("Reconstruction errors:", errors_val)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Reconstruction errors shape: (6,)
Reconstruction errors: [0.00603369 0.00539903 0.00633283 0.00649404 0.00627899 0.00934843]


In [13]:
mean_val_error = np.mean(errors_val)
std_val_error = np.std(errors_val)

threshold = mean_val_error + 2 * std_val_error
print("Threshold:", threshold)


Threshold: 0.009162963


In [ ]:
# Compare similarity using latent features
def calculate_similarity(encoder, input_logo, valid_logos):
    input_features = encoder.predict(input_logo)
    similarities = []
    for valid_logo in valid_logos:
        valid_features = encoder.predict(valid_logo)
        similarity = np.dot(input_features, valid_features.T) / (
            np.linalg.norm(input_features) * np.linalg.norm(valid_features)
        )
        similarities.append(similarity)
    return similarities



In [ ]:
# Example usage
input_logo = ...  # Load input image and preprocess
valid_logos = [...]  # List of preprocessed valid logos
similarities = calculate_similarity(encoder, input_logo, valid_logos)


## Second way to solve the Error: None values not supported.

In [70]:
train_dir = "/home/mahdi/logo_detection_model/autoencoder/train"
val_dir = "/home/mahdi/logo_detection_model/autoencoder/validation"
height = 224
width = 224

def autoencoder_generator(original_generator):
    for batch in original_generator:
        # batch is typically a tuple (images, labels) if class_mode != None
        # or just images if class_mode=None. 
        # We want (images, images).
        yield (batch, batch)

train_datagen = ImageDataGenerator(rescale=1./255)
base_train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(height, width),
    batch_size=32,
    class_mode=None,  # yields just x
    shuffle=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
base_val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(height, width),
    batch_size=32,
    class_mode=None,  # yields just x
    shuffle=False
)

# Wrap the original generator
train_generator = autoencoder_generator(base_train_generator)
val_generator = autoencoder_generator(base_val_generator)

history = autoencoder.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    steps_per_epoch=base_train_generator.samples // base_train_generator.batch_size,
    validation_steps=base_val_generator.samples // base_val_generator.batch_size,
    verbose=1
)

Found 206 images belonging to 1 classes.
Found 52 images belonging to 1 classes.
Epoch 1/50


/home/mahdi/logo_detection_model/.venv/lib/python3.12/site-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 709ms/step - loss: 0.0042 - val_loss: 0.0048
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 709ms/step - loss: 0.0043 - val_loss: 0.0046
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 618ms/step - loss: 0.0041 - val_loss: 0.0046
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 618ms/step - loss: 0.0041 - val_loss: 0.0044
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 616ms/step - loss: 0.0039 - val_loss: 0.0044
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 624ms/step - loss: 0.0039 - val_loss: 0.0043
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 625ms/step - loss: 0.0038 - val_loss: 0.0043
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 703ms/step - loss: 0.0037 - val_loss: 0.0042
Epoch 9/50
2/6 ━━━━━━━━━━━━━━━━━━━━ 2s 662ms/step - loss: 0.0034

KeyboardInterrupt: 